In [23]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [24]:
df = pd.read_csv('./datasets/GP_data_2016_2018_기본변수+타겟+졸업대학+가족.csv')
df.head()

,연도,본분교,국공사립,학위과정유형,주야간,전공계열,전공_소분류_코드,전공_세분류_코드,학교유형,학교소재지역,...,G181G019,G181G033,G181G038,G181P001,G181P004,G181P026,G181P029,G181P035,G181P036,G181P042
0,2018,1,3,3,2,1,C010101,C01010100027,1,1,...,NaN,NaN,NaN,1,NaN,4,4,5,3,NaN
1,2018,1,3,3,1,1,C010101,C01010100013,1,1,...,NaN,NaN,NaN,1,NaN,4,4,4,3,NaN
2,2018,1,3,3,1,1,C010101,C01010100013,1,1,...,1.0,NaN,NaN,1,NaN,6,6,6,5,40.0
3,2018,1,3,3,1,1,C010101,C01010100005,1,8,...,NaN,NaN,NaN,1,NaN,6,6,6,5,NaN
4,2018,1,3,3,1,1,C010101,C01010100005,1,8,...,NaN,NaN,NaN,1,NaN,4,4,6,4,NaN


In [25]:
basic_df = df[['연도','본분교','국공사립','학위과정유형','주야간','전공계열','전공_소분류_코드','전공_세분류_코드','학교유형',
               '학교소재지역','성별','연령','G181SQ001']]
basic_df.head()

,연도,본분교,국공사립,학위과정유형,주야간,전공계열,전공_소분류_코드,전공_세분류_코드,학교유형,학교소재지역,성별,연령,G181SQ001
0,2018,1,3,3,2,1,C010101,C01010100027,1,1,2,23.500000,1
1,2018,1,3,3,1,1,C010101,C01010100013,1,1,1,21.750000,12
2,2018,1,3,3,1,1,C010101,C01010100013,1,1,1,21.916667,1
3,2018,1,3,3,1,1,C010101,C01010100005,1,8,2,25.583333,1
4,2018,1,3,3,1,1,C010101,C01010100005,1,8,1,21.750000,1


In [26]:
# print(basic_df['본분교'].unique())
# print(basic_df['국공사립'].unique())
# print(basic_df['학위과정유형'].unique())
# print(basic_df['주야간'].unique())
# print(basic_df['전공계열'].unique())
print(basic_df['전공_소분류_코드'].nunique())
print(basic_df['전공_세분류_코드'].nunique())

# 전공_소분류_코드 / 전공_세분류_코드 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
basic_df['전공_소분류_코드'] = encoder.fit_transform(basic_df['전공_소분류_코드'])
basic_df['전공_세분류_코드'] = encoder.fit_transform(basic_df['전공_세분류_코드'])

# print(basic_df['전공_소분류_코드'].unique())
# print(basic_df['전공_세분류_코드'].unique())

206
5034


In [27]:
# 연령 -> 소수점 버리기(내림)
import math

age = []

for i in basic_df['연령'].values:
    age.append(math.trunc(i))

basic_df['연령'] = age
print(basic_df['연령'])

0        23
1        21
2        21
3        25
4        21
         ..
54438    27
54439    50
54440    26
54441    27
54442    25
Name: 연령, Length: 54443, dtype: int64


In [32]:
# G181SQ001 -> SQ1. 지난 4주 간 주로 한 일
# 1 : 일하였음 , 2 : 일시휴직 -> 두개 다 취업한 상태로 변경(1)
# 3~15 : 미취업 -> 0으로 변경

# 컬럼명 변경
basic_df.rename(columns={'G181SQ001' : '취업여부'} , inplace=True)


for i in basic_df['취업여부']:
    if 3<= i <=15:
        basic_df['취업여부'].replace(i , 0 , inplace=True)
    elif i == 2:
        basic_df['취업여부'].replace(i , 1 , inplace=True)

print(basic_df['취업여부'].nunique())

2


In [34]:
basic_df.취업여부.value_counts()

# 취업 : 68% 
# 미취업 : 32%
# 모델링 진행시 과적합 방지를 위해 SMOTE 기법 필요

1    37413
0    17030
Name: 취업여부, dtype: int64

In [36]:
basic_df.to_csv('GP_basic+target(3years).csv' , index=False , encoding='utf-8-sig')